In [1]:
import requests
import base64
import os
import mimetypes
from PIL import Image
from datetime import datetime

In [ ]:
# Azure OpenAI API Configuration
AZURE_OPENAI_API_KEY="YOUR_KEY"
AZURE_OPENAI_ENDPOINT="https://YOUR_SERVICE_NAME.openai.azure.com/"
AZURE_OPENAI_API_VERSION="2025-04-01-preview"

# GPT Models
AZURE_OPENAI_TEXT_DEPLOYMENT="gpt-4o-mini"
AZURE_OPENAI_IMAGE_DEPLOYMENT="gpt-image-1"

# Basic Image Generation

In [ ]:
# prompt = f"EcoMealBoxをベースの画像として変更せずに使用します。EcoMealBoxの左下の仕切りに唐揚げ(Karaage)、左上の仕切りにポテトサラダ(PotatoSalad)を追加。その他の仕切りは変更しないでください"
# prompt = f"Use EcoMealBox as base and don't change this figure. Add Karaage to lower-left section of box, PotatoSalad to upper-left. Do not change other sections."
prompt = f"Use EcoMealBox as base and don't change this figure. Add Kara-age to lower-left section of the box. Do not change other sections."


url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
headers = {
    # "Content-Type": "multipart/form-data",
    "api-key": AZURE_OPENAI_API_KEY
}

files = {
    "image":("ECOMealBox", open("./images/input/ECOMealBox.jpg", "rb"), "image/jpeg")
    # "image":("Kara-age", open("./karaage.jpg", "rb"), "image/jpeg"),
    # "image":("PotatoSalad", open("./potatosalad.jpg", "rb"), "image/jpeg"),
    # "image":("MameGohan", open("./mamegohan.jpg", "rb"), "image/jpeg")
} 

data = {
    "prompt": prompt,
    "size": "1024x1024",
    "n": "1"
}

response = requests.post(url, headers=headers, data=data, files=files)
print("response code: " + str(response.status_code))

if response.status_code == 200:
    b64 = response.json()["data"][0]["b64_json"]
    path = f"./images/generated/{dt_str}.jpg"
    with open(path, "wb") as out:
        out.write(base64.b64decode(b64))
    print("Image saved to: " + path)
else:
    print("Error: " + response.text)
    exit(1)

In [18]:
def generate_image(prompt,images,generated_image):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        "api-key": AZURE_OPENAI_API_KEY
    }
    
    data = {
        "prompt": prompt,
        "size": "1024x1024",
        "n": 1
    }

    # files = {
    #     "image": (os.path.basename(image), open(image, "rb"), mimetypes.guess_type(image))
    # }

    files = []
    for image in images:
        if not os.path.exists(image):
            print(f"Error: Image file {image} does not exist.")
            exit(1)
        filename = os.path.basename(image)
        filetype, _ = mimetypes.guess_type(image)
        files.append(("image[]", (filename, open(image, "rb"), filetype)))
    
    response = requests.post(url, headers=headers, data=data, files=files)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        with open(generated_image, "wb") as out:
            out.write(base64.b64decode(b64))
        print("Image saved to: " + generated_image)
    else:
        print("Error: " + response.text)
        exit(1)

# Obento generation

In [ ]:
# dt_str = datetime.now().strftime("%Y%m%d_%H%M%S")

def dt_str():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

## step-by-step

In [ ]:
prompt = f"Use EcoMealBox as base and don't change this figure. Add Potato-salad to upper-left section of the box. Do not change other sections."
images = ["./images/input/ECOMealBox_01.jpg"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

In [ ]:
prompt = f"Use EcoMealBox as base and don't change this figure. Add greenpea-mixed-rice to upper-right section of the box. Do not change other sections."
image = ["./images/input/ECOMealBox_02.jpg"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, image, generated_image)

In [ ]:
prompt = f"Use EcoMealBox as base and don't change this figure. Add tomato-salad to lower-right section of the box. Do not change other sections."
image = ["./images/input/ECOMealBox_03.jpg"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, image, generated_image)

## mixed-at-once

In [ ]:
prompt = f"Use EcoMealBox as base and don't change this figure. Add Kara-age to lower-left, Potato-salad to upper-left, greenpea-mixed-rice to upper-right, tomato-salad to lower-right section of the box. Do not change other."
image = ["./images/input/ECOMealBox.jpg"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, image, generated_image)

## using image input

In [26]:
prompt = f"Image of EcoMealBox with food as guided. Don't change EcoMealBox figure. Do not change anything with EchoBox other than guided. Here is guide: Add Shushoku to lower-left of the box. Other area remains unchanged."
images = ["./images/input/ECOMealBox.jpg", "./images/input/Shushoku_03.png"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_141510.jpg


In [29]:
prompt = f"Image of EcoMealBox with food as guided. Don't change EcoMealBox figure. Do not change anything with EchoBox other than guided. No label on image. Here is guide: Add Shushoku to Section01 of the box. Other area remains unchanged."
images = ["./images/input/ECOMealBox_00.jpg", "./images/input/Shushoku_03.png"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_153304.jpg


In [32]:
prompt = f"Image of EcoMealBox with food as guided. Don't change EcoMealBox figure. Do not change anything with EchoBox other than guided. No label on image. Here is guide: Add Shushoku to Section03-01, Shusai to Section01 of the box. Other area remains unchanged."
images = ["./images/input/ECOMealBox_00.jpg", "./images/input/Shushoku_03.png", "./images/input/Shusai_01.png"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_153555.jpg


In [ ]:
prompt = f"Image of EcoMealBox with food as guided. Add foods to sections aslabeled in ECOMealBox_00. Don't change EcoMealBox figure. Do not change anything with EchoBox other than guided. No label on image. Here is guide: Add Shushoku to Section03-01, Shusai to Section01 of the box. Other area remains unchanged."
images = ["./images/input/ECOMealBox.jpg", "./images/input/ECOMealBox_00.jpg", "./images/input/Shushoku_03.png", "./images/input/Shusai_01.png"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_154112.jpg


In [37]:
# prompt update by AI
prompt = f"Generate an image based on EcoMealBox, using the layout and section labels from ECOMealBox_00. Do not alter the shape, design, or appearance of the EcoMealBox itself. Do not modify any areas except as instructed. Do not add any labels or text to the image. Follow this guide for food placement: place Shushoku in Section03-01 and Shusai in Section01. Leave all other sections unchanged."
images = ["./images/input/ECOMealBox.jpg", "./images/input/ECOMealBox_00.jpg", "./images/input/Shushoku_03.png", "./images/input/Shusai_01.png"]
generated_image = f"./images/generated/{dt_str}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_154532.jpg


In [44]:
# 3 items
prompt = f"Generate an image based on EcoMealBox, using the layout and section labels from ECOMealBox_00. Do not alter the shape, design, or appearance of the EcoMealBox itself. Do not modify any areas except as instructed. Do not add any labels or text to the image. Follow this guide for food placement: place Shusai in Section03-01 and Section03-02, Shushoku in Section01, and Fukusai in Section02. Leave all other sections unchanged."
images = ["./images/input/ECOMealBox.jpg", "./images/input/ECOMealBox_00.jpg", "./images/input/Shushoku_03.png", "./images/input/Shusai_01.png","./images/input/Fukusai_01.png"]
generated_image = f"./images/generated/{dt_str()}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_161007.jpg


In [48]:
# 3 items without section guidance
prompt = f"Generate an image based on EcoMealBox, using the layout from ECOMealBox. Do not alter the shape, design, or appearance of the EcoMealBox itself. Do not modify any areas except as instructed. Do not add any labels or text to the image. Follow this guide for food placement: place Shusai in first-largest section, Shushoku in second-largest, and Fukusai in other one section. Leave all other sections unchanged."
images = ["./images/input/ECOMealBox.jpg", "./images/input/Shushoku_03.png", "./images/input/Shusai_01.png","./images/input/Fukusai_01.png"]
generated_image = f"./images/generated/{dt_str()}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/20250611_162738.jpg


# Labeling Box Sections

In [47]:
prompt = f"Analyze the provided EcoMealBox image and accurately detect each section within the box. Clearly label each detected section with its corresponding section name, such as Section01, Section02, Section03-01, Section03-02, etc. Ensure the labels are easy to read and are placed directly on each section. Do not alter the shape, design, or appearance of the EcoMealBox itself, and do not add any other elements to the image."
images = ["./images/input/ECOMealBox.jpg"]
generated_image = f"./images/generated/EcoMealBox_sectioned_{dt_str()}.jpg"
generate_image(prompt, images, generated_image)

response code: 200
Image saved to: ./images/generated/EcoMealBox_sectioned_20250611_162349.jpg


# Masking

In [ ]:
prompt = f"Paint with black only upper-left section of the box."
image = "./ECOMealBox.jpg"
generated_image = "./mask.jpg"
generate_image(prompt, image, generated_image)

In [ ]:
def create_mask(mask,mask_alpha):
    from PIL import Image
    from io import BytesIO

    mask = Image.open(mask).convert("L")
    mask_rgba = mask.convert("RGBA")

    mask_rgba.putalpha(mask)

    buf = BytesIO()
    mask_rgba.save(buf, format="PNG")
    mask_bytes = buf.getvalue()

    with open(mask_alpha, "wb") as f:
        f.write(mask_bytes)

In [ ]:
create_mask("./mask.jpg", "./mask_alpha.png")

In [ ]:
def generate_image_with_mask(prompt,image,mask,generated_image):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        "api-key": AZURE_OPENAI_API_KEY
    }
    
    data = {
        "prompt": prompt,
        "size": "1024x1024",
        "n": 1
    }

    files = {
        "image": ("image", open(image, "rb"), "image/jpeg"),
        "mask": ("mask", open(mask, "rb"), "image/png")
    }
    
    response = requests.post(url, headers=headers, data=data, files=files)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        with open(generated_image, "wb") as out:
            out.write(base64.b64decode(b64))
    else:
        print("Error: " + response.text)
        exit(1)

In [ ]:
prompt = f"add potato salad to upper-left of meal box. do not change figure of meal box nor other sections."
image = "./ECOMealBox.jpg"
mask = "./mask_alpha.png"
generated_image = "./generated_image_with_mask.jpg"
generate_image_with_mask(prompt, image, mask, generated_image)

In [ ]:
def generate_image_with_images_mask(prompt, image_base, image_sub, mask, generated_image):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        "api-key": AZURE_OPENAI_API_KEY
    }
    
    data = {
        "prompt": prompt,
        "size": "1024x1024",
        "n": 1
    }

    files = {
        "image": (os.path.basename(image_base), open(image_base, "rb"), "image/png"),
        "image": (os.path.basename(image_sub), open(image_sub, "rb"), "image/png"),
        "mask": ("mask", open(mask, "rb"), "image/png")
    }
    
    response = requests.post(url, headers=headers, data=data, files=files)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        with open(generated_image, "wb") as out:
            out.write(base64.b64decode(b64))
    else:
        print("Error: " + response.text)
        exit(1)

In [ ]:
from PIL import Image

def convert_jpeg_to_png_alpha(mask, mask_alpha):
    image = Image.open(mask).convert("RGBA")
    alpha = Image.new("L", image.size, 255)
    image.putalpha(alpha)
    image.save(mask_alpha, format="PNG")


In [ ]:
convert_jpeg_to_png_alpha("./mask.jpg", "./mask_alpha.png")

In [ ]:
# prompt = f"Add karaage to upper-left of meal box. Do not change box figure. Do not add anything others."
prompt = f"唐揚げをお弁当箱に追加"
image_base = "./ECOMealBox.png"
image_sub = "./karaage.png"
mask = "./mask_alpha.png"
generated_image = "./generated_image_with_images_mask.jpg"
generate_image_with_images_mask(prompt, image_base, image_sub, mask, generated_image)

In [ ]:
def generate_mask(prompt,image,mask):
    url = f"{AZURE_OPENAI_ENDPOINT}openai/deployments/{AZURE_OPENAI_IMAGE_DEPLOYMENT}/images/edits?api-version={AZURE_OPENAI_API_VERSION}"
    headers = {
        "api-key": AZURE_OPENAI_API_KEY
    }
    
    data = {
        "prompt": prompt,
        "size": "1024x1024",
        "n": 1
    }

    files = {
        "image": ("image", open(image, "rb"), "image/jpeg")
    }
    
    response = requests.post(url, headers=headers, data=data, files=files)
    print("response code: " + str(response.status_code))

    if response.status_code == 200:
        b64 = response.json()["data"][0]["b64_json"]
        with open(mask, "wb") as out:
            out.write(base64.b64decode(b64))

        image = Image.open(mask).convert("RGBA")
        alpha = Image.new("L", image.size, 255)
        image.putalpha(alpha)
        image.save(mask, format="PNG")
    else:
        print("Error: " + response.text)
        exit(1)

In [ ]:
prompt = f"Paint with black only upper-left section of the box and save as mask."
generate_mask(prompt, "./ECOMealBox.png", "./mask_alpha_black.png")

In [ ]:
prompt = f"ECOMealBoxの左下だけにmamegohanが入っている"
image_base = "./ECOMealBox.png"
image_sub = "./mamegohan.png"
mask = "./mask_alpha_white.png"
generated_image = "./generated_image_with_images_mask.jpg"
generate_image_with_images_mask(prompt, image_base, image_sub, mask, generated_image)

In [ ]:
prompt = f"Paint with black only upper-left section of the box and save as mask."
generate_mask(prompt, "./images/Container_01.png", "./images/Container_01_mask.png")

In [ ]:
prompt = prompt = f"Paint with black only upper-left section of Container_01 and save as mask."
image_base = "./images/Container_01.png"
mask = "./images/Container_01_mask.png"

generate_mask(prompt, image_base, mask)

In [ ]:
prompt = f"Shusai_01 appears in Container_01"
image_base = "./images/Container_01.png"
image_sub = "./images/Shusai_01.png"
mask = "./images/Container_01_mask.png"
generated_image = "./images/generated/" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".jpg"

generate_image_with_images_mask(prompt, image_base, image_sub, mask, generated_image)